In [1118]:
from collections import defaultdict
import numpy as np
import os


WORDTAG = 'WORDTAG'
GRAM1 = '1-GRAM'
GRAM2 = '2-GRAM'
GRAM3 = '3-GRAM'
RARE = '_RARE_'
RARE_THRESHOLD = 5
# A I-LOC might be followed by B-LOC if a new location follow one immediately after another one                                                                                                              
# There are 9 tags to consider                                                                                                                                                                               
TAGS = [
    'I-PER',
    'I-ORG',
    'I-LOC',
    'I-MISC',
    'B-PER',
    'B-ORG',
    'B-LOC',
    'B-MISC',
    'O'
]
# The start and stop symbols                                                                                                                                                                                 
START = '*'
STOP = 'STOP'

FILL_IN = '_FILL_IN_'

In [1157]:
# q_counts[(u, v, w)] = count(u, v, w)
# q_counts[[u, v]] = count(u, v)
# e_counts[(u, x)] = count(u, x)
def get_q_e_counts(counts_file_name = 'ner.counts'):
    f = open(counts_file_name)

    q_counts = defaultdict(int)
    e_counts = defaultdict(int)

    for line in f:
        parts = line.strip().split(" ")
        count = int(parts[0])
        if parts[1] == 'WORDTAG':
            tag = parts[2]
            word = parts[3]
            e_counts[(tag, word)] = count
            e_counts[tag] = e_counts.get(tag, 0) + count
        elif parts[1] == '3-GRAM':
            u = parts[2]
            v = parts[3]
            w = parts[4]
            q_counts[(u, v, w)] = count
        elif parts[1] == '2-GRAM':
            u = parts[2]
            v = parts[3]
            q_counts[(u, v)] = count

    f.close()

    return q_counts, e_counts

In [1158]:
e_counts = get_q_e_counts()[1]
print(len(e_counts))

24780


In [1159]:
# This transforms the data into one involving the rare words                                                                                                                                                 
# We then run the count_freqs.py utility to get the new counts across the corpus                                                                                                                             
def transform_data(e_counts):
    """
    Input:
        e_counts: A dictionary with counts(y, x) and counts(y)
    Output:
        Nothing; write to g
    """

    f = open('ner_train.dat', 'r')
    g = open('ner_train_rare.dat', 'w')

    # Identify rare words
    word_tracker = defaultdict(int)
    for key, count in e_counts.items():
        if isinstance(key, tuple):
            (tag, word) = key
            word_tracker[word] += count
    
    rare_words = {word for word, count in word_tracker.items() if count < RARE_THRESHOLD}

    # Read the input file, replace rare words and write to the output file
    for line in f:
        parts = line.strip().split()
        if len(parts) > 1:  # Ensure it's not an empty line
            word, tag = parts
            if word in rare_words:
                g.write(f"{RARE} {tag}\n")
            else:
                g.write(line)
        else:
            g.write("\n")  # Keep sentence boundaries

    f.close()
    g.close()

In [1121]:
# These are the probabilities e(x_t | y_t)                                                                                                                                                                   
def get_emission(y, x, e_counts, x_counts):
    """
    Input:
        y: A tag
        x: A word
        e_counts: A dictionary with counts(y, x) and counts(y)
        x_counts: A dictionary with counts(x)
    Output:
        The probabilty e(x|y) or e(RARE|y) is x is rare
        This is vartheta(x | y) in the lecture
    """
    if y not in e_counts:
        return 0
    # If a rare word, return e(RARE | y)
    if x_counts[x] < RARE_THRESHOLD:
        return e_counts[(y, RARE)] / e_counts[y]
    # Otherwise, return e(x | y)                                                                                                                                                                             
    return e_counts[(y, x)] / e_counts[y]

In [1122]:
# Not that for the baseline decoder we don't need Dynamic Programming
# We have max_{y1, ..., YT} = max_{y1}(e(x1|y1))...max_{yT}(e(xT|yT))
def baseline_ner_tagger(
        counts_file_name = 'ner_rare.counts'
):
    """
    Input:
        counts_file_name: The counts file we use
    Output:
        Nothing; write to a new file "x, y, log(e(x|y))" where y is the optimal tag for x
    """
    f = open('ner_dev.dat', 'r')
    g = open('ner_dev.baseline_predictions', 'w')

    _, e_counts = get_q_e_counts(counts_file_name)
    
    # Get the counts per word; this is used to get the rare words                                                                                                                                            
    # What words need to be replaced with a rare word?
    # Note that we do here is take all counts of (u, x) for all u to get the count for x
    x_counts = defaultdict(int)
    _, e_ori = get_q_e_counts('ner.counts')
    for key, count in e_ori.items():
        if isinstance(key, tuple):
            (tag, word) = key
            x_counts[word] += count          

    for l in f:
        x = l.strip()
        if not x:
            g.write("\n")
        else:
            p_best = -float('inf')
            y_best = None
            for tag in TAGS:
                p = get_emission(tag, x, e_counts, x_counts)
                if p > p_best:
                    p_best = p
                    y_best = tag
            g.write('{} {} {}\n'.format(x, y_best, np.log(p_best)))
    f.close()
    g.close()

In [1123]:
# These are the probabilities p(y_t | y_{t-1}, y_{t-2})                                                                                                                                                      
def get_transition(y1, y2, y3, q_counts):
    """
    Input:
        y1: The tag two away from the output tag
        y2: The tag right before the output tag
        y3: The output tag
        q_counts: The counts we need for two or 3 tags being seen together
    Output:
        q(w | v, u) which is theta(w | v, u) in the lecture
    """
    if (y1, y2) not in q_counts:
        return 0
    return q_counts.get((y1, y2, y3), 0)/q_counts[(y1, y2)]

In [1140]:
def hmm_ner_tagger(
        counts_file_name = 'ner_rare.counts'
):
    """
    Input:
        counts_file_name: The counts file we use
    Output:
        Nothing; write to a new file "x_t, y_t, log(pi(t, y_{t-1}, y_t))" where y_t is the optimal tag for x_t
        Note that {y_t} is the optimal sequence here, computed by Dynamic Programming
    """
    f = open('ner_dev.dat', 'r')
    g = open('ner_dev.hmm_predictions', 'w')

    q_counts, e_counts = get_q_e_counts(counts_file_name)

    # Get the counts per word; this is used to identify                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
    x_counts = defaultdict(int)
    _, e_ori = get_q_e_counts('ner.counts')
    for key, count in e_ori.items():
        if isinstance(key, tuple):
            tag, word = key
            x_counts[word] += count

    # Can use log probabilities here                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    # Reset all variables                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
    pi = defaultdict(float)
    bp = defaultdict(str)
    pi[(0, START, START)] = 1.0
    T = 0
    xT = []
    for l in f:
        if not l or l == '\n':
            # We have an empty line; if xT has data in it then decode it by working backwords                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
            if xT:
                # Define the default values of v and w here                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
                pi_max = float('-inf')
                v_max = None
                w_max = None

                # Here we define the tag sequence of v and w
                # pi(T, v, w) + np.log(get_transition(v, w, STOP, q)) is what we want to maximize
                # We need v and w and from this we need to work back
                v_tags = [START] if T == 1 else TAGS
                w_tags = TAGS

                for v in v_tags:
                    for w in w_tags:
                        current_value = pi[(T, v, w)] + np.log(get_transition(v, w, STOP, q_counts))
                        if current_value > pi_max:
                            pi_max = current_value
                            v_max = v
                            w_max = w
                
                # Set yT be the sequence [v_max, w_max] if T > 1 and [w_max] otherwise
                yT = [v_max, w_max] if T > 1 else [w_max]

                """
                Use backpointers to get the sequence we seek 
                This is the highest probability tag sequence (y1,..., yT)
                Remember we just found v_max and w_max and we have 
                pi(T, v_max, w_max) = np.log(e(xT | w_max)) + \max_{u}(q(w_max | v_max, y)*pi(T-1, u, v_max))
                We need u, which should be u_max = bp[(T, v_max, w_max)]
                We append this to yT to get [u_max, v_max, w_max]
                We continue this process on until T = 1 (use a loop)
                """
    
                for t in range(T, 2, -1): 
                    u_max = bp[(t, yT[0], yT[1])]
                    yT.insert(0, u_max)
                log_pT = []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
                assert(T == len(xT))
                assert(len(yT) == len(xT))
                
                """
                We want to get the log probability of the sequence
                For example, when we are at x1 this is
                np.log(q(y1, START, START)) + np.log(e(x1|y1))
                """
                for t in range(len(xT)):
                    if t == 0:
                        log_pT.append(np.log(get_transition(START, START, yT[t] , q_counts)) + np.log(get_emission(yT[t], xT[t], e_counts, x_counts)))
                    elif t == 1:
                        log_pT.append(np.log(get_transition(START, yT[t-1], yT[t] , q_counts)) + np.log(get_emission(yT[t], xT[t], e_counts, x_counts)))
                    else:
                        log_pT.append(np.log(get_transition(yT[t-2], yT[t-1], yT[t] , q_counts)) + np.log(get_emission(yT[t], xT[t], e_counts, x_counts)))
                for xt, yt, log_pt in zip(xT, yT, log_pT):
                    g.write('{} {} {}\n'.format(xt, yt, log_pt))
                g.write('\n')


            # Reset all variables
            # For the next sentence, we'll append words as we see them and compute these 
            pi = defaultdict(float)
            bp = defaultdict(str)
            pi[(0, START, START)] = 1.0
            T = 0
            xT = []
        else:
            # This is the forward step of Dynamic Programming, where we go from T-1 -> T
            l = l.strip().split(' ')
            #print(l)
            T += 1
            xt = l[-1]
            xT.append(xt)

            # q(w | v, u)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
            # What can u be? Consider q(w | v, u) when T = 1 or T = 2 vs more                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
            u_tags = [START] if T <= 2 else TAGS
            # What can v be? Consider q(w | v, u) when T = 1 [Ovs more                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
            v_tags = [START] if T == 1 else TAGS
            # What can w be? w can only be a true TAG, never START                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
            w_tags = TAGS

            """                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
            For this we use the recursion below:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
            v, w in v_tags, w_tags while u is over u_tags

            The probability recursion:
            pi(t, v, w) = e(xt | w) max_{u}{q(w | v, u) * pi(t-1, u, v)}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

            Becomes the log recursion:
            pi(t, v, w) = log e(xt | w) + max_{u}{log q(w | v, u)  + pi(t-1, u, v)}                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

            We use logs below to make it easier and avoid overflow                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
            """
            for v in v_tags:
                for w in w_tags:
                    # e(x | w); this term is not in the max                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
                    e_temp = np.log(get_emission(w, xt, e_counts, x_counts))

                    # pi(t, v, w) = log e(xt | w)  + max_{u}{log q(w | v, u)  + pi(t-1, u, v)}                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                    pi_max = float('-inf')
                    u_max = None

                    # Do the max with respect to u                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
                    for u in u_tags:
                        q_value = np.log(get_transition(u, v, w, q_counts))
                        current_value = q_value + e_temp + pi[(T-1, u, v)]
                        if current_value > pi_max:
                            pi_max = current_value
                            u_max = u

                    # The arg max of max_{u}{log q(w | v, u)  + pi(t-1, u, v)}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                    bp[(T, v, w)] = u_max

                    # The log probability of ending in (v, w) at time T                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
                    pi[(T, v, w)] = pi_max


# Run code below

In [1160]:
# This gets the number of lines in new_train.dat
!wc -l ner_train.dat

  217662 ner_train.dat


In [1161]:
!python count_freqs.py ner_train.dat > ner.counts

In [1162]:
!head ner.counts

24 WORDTAG I-ORG EU
1 WORDTAG O rejects
84 WORDTAG I-MISC German
30 WORDTAG O call
3382 WORDTAG O to
5 WORDTAG O boycott
78 WORDTAG I-MISC British
3 WORDTAG O lamb
7362 WORDTAG O .
31 WORDTAG I-PER Peter


In [1163]:
!wc -l ner.counts

   24968 ner.counts


In [1164]:
 # This does the flow of everything, you might want to comment out certain parts                                                                                                                                                                                                       
q_counts, e_counts = get_q_e_counts('ner.counts')

In [1165]:
e_counts['I-ORG']

10001

In [1166]:
len(e_counts)

24780

In [1167]:
# Get the new data and replace all rare words with _RARE_                                                                                                                                                                                                                             
transform_data(e_counts)

18710


In [1168]:
# Should be the same number of lines as above
!wc -l ner_train_rare.dat

  217662 ner_train_rare.dat


In [1169]:
# Run the count_freqs helper again to get the new counts                                                                                                                                                                                                                              
# This requires a run outside of this                                                                                                                                                                                                                                                 
!python count_freqs.py ner_train_rare.dat > ner_rare.counts

In [1170]:
# Many words will get mapped to _RARE_, so it is fairly simple
!wc -l ner_rare.counts

    5959 ner_rare.counts


In [1152]:
# Get the rare counts for each word
# These will allow us to get the new probabilities
q_counts, e_counts = get_q_e_counts('ner_rare.counts')

In [1153]:
# Get baseline model's performance                                                                                                                                                                                                                                                            
baseline_ner_tagger('ner_rare.counts')

In [1154]:
# This evaluates the baseline tagger
!python eval_ne_tagger.py ner_dev.key ner_dev.baseline_predictions

Found 14043 NEs. Expected 5931 NEs; Correct: 3117.

	 precision 	recall 		F1-Score
Total:	 0.221961	0.525544	0.312106
PER:	 0.435451	0.231230	0.302061
ORG:	 0.475936	0.399103	0.434146
LOC:	 0.147750	0.870229	0.252612
MISC:	 0.491689	0.610206	0.544574


In [1155]:
# Get HMM model's performance                                                                                                                                                                                                                                                                 
hmm_ner_tagger('ner_rare.counts')

<ipython-input-1140-662f97edec31>:129: RuntimeWarning: divide by zero encountered in log
  e_temp = np.log(get_emission(w, xt, e_counts, x_counts))
<ipython-input-1140-662f97edec31>:137: RuntimeWarning: divide by zero encountered in log
  q_value = np.log(get_transition(u, v, w, q_counts))
<ipython-input-1140-662f97edec31>:48: RuntimeWarning: divide by zero encountered in log
  current_value = pi[(T, v, w)] + np.log(get_transition(v, w, STOP, q_counts))


In [1156]:
# This evaluates the HMM tagger; performance should be about double that of the baseline
!python eval_ne_tagger.py ner_dev.key ner_dev.hmm_predictions

Found 4704 NEs. Expected 5931 NEs; Correct: 3649.

	 precision 	recall 		F1-Score
Total:	 0.775723	0.615242	0.686225
PER:	 0.763928	0.596844	0.670128
ORG:	 0.611855	0.478326	0.536913
LOC:	 0.876458	0.696292	0.776056
MISC:	 0.830065	0.689468	0.753262
